# Diamond Company Analytics Challenge

**Time:** 35 minutes

**Scenario:** You are the new data analyst at a prestigious diamond company. The CEO has called an urgent meeting and needs answers to critical business questions about the inventory.

**Your Mission:** Analyze the diamond inventory data and provide insights to guide strategic decisions.

In [ ]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


## Section 1: Initial Inventory Assessment (5 minutes)

The CEO walks in: "Before we dive into specifics, I need to understand what we're working with. Give me the basics about our inventory database."

### Question 1: "How extensive is our inventory database?"

The CEO needs to know the scale of data you're analyzing.

In [ ]:
# Your code here:
num_rows, num_cols = df.shape

print(f"Our inventory database contains {num_rows:,} diamonds.")


Our inventory database contains 6,000 diamonds.


### Question 2: "What information do we track about each diamond?"

List all the characteristics we record.

In [ ]:
# Your code here:

print(df.columns)


characteristics_list = ", ".join(df.columns)

print(f"For each diamond, the characteristics we track are: {characteristics_list}.")



['Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price']
For each diamond, the characteristics we track are: Carat Weight, Cut, Color, Clarity, Polish, Symmetry, Report, Price.


### Question 3: "When I look at one line in this database, what am I looking at?"

Explain in business terms:

**Your answer:** Each row represents: all the characteristics of a single diamond in our inventory.

## Section 2: Strategic Business Questions (20 minutes)

The CEO continues: "Now let's get to the important questions. Our board meeting is tomorrow and I need concrete numbers."

### Question 4: "We're considering expanding our storage facilities. How many total carats do we have in inventory? I need to know if our vaults can handle the weight."

Think about what measurement would help answer this.

In [ ]:
# Your code here:

total_carats = df['Carat Weight'].sum()
print(f"The total weight of diamonds in inventory is {total_carats:,.2f} carats.")


The total weight of diamonds in inventory is 8,007.12 carats.


### Question 5: "Our new marketing campaign claims we have 'the widest variety of cuts in the industry.' Can you verify this?"

The CEO wants to know about variety, not quantity.

In [ ]:
# Your code here:
unique_cuts = df['Cut'].unique().sort().to_list()
num_unique_cuts = len(unique_cuts)

print(f"We have {num_unique_cuts} unique cuts in our inventory: {unique_cuts}.")


We have 5 unique cuts in our inventory: ['Fair', 'Good', 'Ideal', 'Signature-Ideal', 'Very Good'].


### Question 6: "I'm meeting with our insurance company. They need to know our price exposure - what's our most and least valuable diamond?"

In [ ]:
# Your code here:

most_valuable = df.sort("Price", descending=True).select(df.columns).head(1)
least_valuable = df.sort("Price").select(df.columns).head(1)

print("Most valuable diamond:")
print(f"The most valuable diamond costs {most_valuable['Price'].item()}")

print("\nLeast valuable diamond:")
print(f"The least valuable diamond costs {least_valuable['Price'].item()}")

Most valuable diamond:
The most valuable diamond costs 101561

Least valuable diamond:
The least valuable diamond costs 2184


### Question 7: "Which color grades should we focus on in our premium line? I need typical prices across our color spectrum."

The CEO wants to understand pricing patterns.

In [ ]:
# Your code here:

avg_price_by_color = (
    df.group_by("Color")
      .agg(pl.col("Price").mean().alias("avg_price"))
      .sort("avg_price", descending=True)
)

print(avg_price_by_color)

shape: (6, 2)
┌───────┬──────────────┐
│ Color ┆ avg_price    │
│ ---   ┆ ---          │
│ str   ┆ f64          │
╞═══════╪══════════════╡
│ D     ┆ 15255.783661 │
│ F     ┆ 12712.241856 │
│ G     ┆ 12520.050633 │
│ E     ┆ 11539.190231 │
│ H     ┆ 10487.347544 │
│ I     ┆ 8989.636364  │
└───────┴──────────────┘


**Reflection:** After analyzing by color, how has your view of the data changed? What does each row in your result represent now?

**Your answer:** New grain = Each row now represents a color grade and the typical price for that color across all diamonds in inventory, rather than a single diamond

## Section 3: Advanced Strategic Analysis (10 minutes)

The CEO leans forward: "These next questions will shape our strategy for next year."

### Question 8: "We're thinking of specializing in certain clarity grades. For our investor deck, I need a complete breakdown showing: how many diamonds we have in each clarity grade, their typical price, and the total carats we're holding for each grade."

In [ ]:
# Your code here:

clarity_summary = (
    df.group_by("Clarity")
      .agg([
          pl.len().alias("num_diamonds"),
          pl.col("Price").mean().alias("avg_price"),
          pl.col("Carat Weight").sum().alias("total_carats")
      ])
)

print(clarity_summary)

shape: (7, 4)
┌─────────┬──────────────┬──────────────┬──────────────┐
│ Clarity ┆ num_diamonds ┆ avg_price    ┆ total_carats │
│ ---     ┆ ---          ┆ ---          ┆ ---          │
│ str     ┆ u32          ┆ f64          ┆ f64          │
╞═════════╪══════════════╪══════════════╪══════════════╡
│ VVS2    ┆ 666          ┆ 14142.177177 ┆ 876.58       │
│ VVS1    ┆ 285          ┆ 16845.680702 ┆ 389.43       │
│ FL      ┆ 4            ┆ 63776.0      ┆ 7.87         │
│ IF      ┆ 219          ┆ 22105.844749 ┆ 316.0        │
│ VS2     ┆ 1575         ┆ 11809.053333 ┆ 2170.81      │
│ SI1     ┆ 2059         ┆ 8018.864012  ┆ 2563.69      │
│ VS1     ┆ 1192         ┆ 13694.113255 ┆ 1682.74      │
└─────────┴──────────────┴──────────────┴──────────────┘


### Question 9: "My luxury competitor claims they have the best prices for every cut-color combination. Show me our pricing structure across these combinations - I want to see where we're competitive."

In [ ]:
# Your code here:

pricing_matrix = (
    df.group_by(["Cut", "Color"])
      .agg(pl.col("Price").mean().alias("avg_price"))
      .sort(["Cut", "Color"])
)

print(pricing_matrix)



shape: (30, 3)
┌───────────┬───────┬──────────────┐
│ Cut       ┆ Color ┆ avg_price    │
│ ---       ┆ ---   ┆ ---          │
│ str       ┆ str   ┆ f64          │
╞═══════════╪═══════╪══════════════╡
│ Fair      ┆ D     ┆ 6058.25      │
│ Fair      ┆ E     ┆ 5370.625     │
│ Fair      ┆ F     ┆ 6063.625     │
│ Fair      ┆ G     ┆ 7345.52381   │
│ Fair      ┆ H     ┆ 5908.5       │
│ …         ┆ …     ┆ …            │
│ Very Good ┆ E     ┆ 12101.910217 │
│ Very Good ┆ F     ┆ 12413.905495 │
│ Very Good ┆ G     ┆ 12354.013841 │
│ Very Good ┆ H     ┆ 10056.106132 │
│ Very Good ┆ I     ┆ 8930.031332  │
└───────────┴───────┴──────────────┘


**Reflection:** How specific is your analysis now? What level of detail does each row represent?

**Your answer:** New grain = Each row represents a unique combination of cut and color, showing the typical price for all diamonds that share that combination

### Question 10: URGENT - "Our biggest client is interested in purchasing entire inventory segments. They asked: 'Which color-cut combinations represent your highest total inventory value?' Show me the top 5 most valuable inventory segments by total worth, and include how many diamonds are in each segment."

This is critical for tomorrow's negotiation.

In [ ]:
# Your code here:

segment_value = (
    df.group_by(["Cut", "Color"])
      .agg([
          pl.col("Price").sum().alias("total_value"),
          pl.len().alias("num_diamonds")
      ])
      .sort("total_value", descending=True)
      .head(5)
)

print(segment_value)


shape: (5, 4)
┌───────────┬───────┬─────────────┬──────────────┐
│ Cut       ┆ Color ┆ total_value ┆ num_diamonds │
│ ---       ┆ ---   ┆ ---         ┆ ---          │
│ str       ┆ str   ┆ i64         ┆ u32          │
╞═══════════╪═══════╪═════════════╪══════════════╡
│ Ideal     ┆ G     ┆ 9363514     ┆ 690          │
│ Very Good ┆ G     ┆ 7140620     ┆ 578          │
│ Very Good ┆ F     ┆ 5648327     ┆ 455          │
│ Ideal     ┆ F     ┆ 5346782     ┆ 363          │
│ Ideal     ┆ H     ┆ 5279687     ┆ 458          │
└───────────┴───────┴─────────────┴──────────────┘


## Debrief

Congratulations! You've just completed your first day as the diamond company's data analyst.

Remember:
- CEOs speak in business language, not technical terms
- Always understand what each row represents before analyzing
- Business questions contain clues about which operations to use